In [ ]:
import struct

# -------------------------------------------------------------------
# Função para ler imagem BMP (24 bits) sem bibliotecas externas - No Colab - Arquivos → Upload → selecione sua imagem (ex: lena.png)
# -------------------------------------------------------------------
def ler_bmp(caminho):
    with open(caminho, "rb") as f:
        header = f.read(54)  # Cabeçalho BMP

        largura = struct.unpack("<I", header[18:22])[0]
        altura  = struct.unpack("<I", header[22:26])[0]
        bpp     = struct.unpack("<H", header[28:30])[0]

        if bpp != 24:
            raise ValueError("Use apenas imagens BMP 24 bits!")

        imagem = []
        padding = (4 - (largura * 3) % 4) % 4

        for _ in range(altura):
            linha = []
            for _ in range(largura):
                b, g, r = struct.unpack("BBB", f.read(3))
                linha.append((r, g, b))
            f.read(padding)
            imagem.append(linha)

        return imagem, largura, altura


# -------------------------------------------------------------------
# Converter RGB → Cinza (Python puro)
# -------------------------------------------------------------------
def rgb_para_cinza(img):
    cinza = []
    for linha in img:
        linha_cinza = []
        for (r, g, b) in linha:
            nivel = int(0.299*r + 0.587*g + 0.114*b)
            linha_cinza.append(nivel)
        cinza.append(linha_cinza)
    return cinza


# -------------------------------------------------------------------
# Converter Cinza → Preto e Branco
# -------------------------------------------------------------------
def cinza_para_pb(img, limiar=128):
    pb = []
    for linha in img:
        nova = []
        for px in linha:
            nova.append(255 if px >= limiar else 0)
        pb.append(nova)
    return pb


# -------------------------------------------------------------------
# Salvar imagem BMP 8 bits (tons de cinza ou PB)
# -------------------------------------------------------------------
def salvar_bmp_8bits(caminho, img):
    altura = len(img)
    largura = len(img[0])

    with open(caminho, "wb") as f:
        # Cabeçalho BMP básico
        tamanho_arquivo = 54 + largura * altura
        header = bytearray(54)

        # Assinatura
        header[0:2] = b"BM"
        struct.pack_into("<I", header, 2, tamanho_arquivo)
        struct.pack_into("<I", header, 10, 54)
        struct.pack_into("<I", header, 14, 40)
        struct.pack_into("<I", header, 18, largura)
        struct.pack_into("<I", header, 22, altura)
        struct.pack_into("<H", header, 26, 1)
        struct.pack_into("<H", header, 28, 8)  # 8 bits
        struct.pack_into("<I", header, 34, largura * altura)

        f.write(header)

        # Paleta (256 tons)
        for i in range(256):
            f.write(bytes([i, i, i, 0]))

        # Imagem
        for linha in img:
            f.write(bytes(linha))


# -------------------------------------------------------------------
# PROCESSO COMPLETO
# -------------------------------------------------------------------

# ➜ Coloque aqui o nome do arquivo enviado no Colab
arquivo = "/content/Violao.bmp"

# 1. Ler imagem colorida
img_colorida, w, h = ler_bmp(arquivo)

# 2. Converter para cinza
img_cinza = rgb_para_cinza(img_colorida)
salvar_bmp_8bits("resultado_cinza.bmp", img_cinza)

# 3. Converter para preto e branco
img_pb = cinza_para_pb(img_cinza)
salvar_bmp_8bits("resultado_pb.bmp", img_pb)

print("Processo concluído!")

Processo concluído!


In [ ]:
# Baixando imagens no colab

from google.colab import files
files.download("resultado_cinza.bmp")
files.download("resultado_pb.bmp")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Violao.bmp to Violao (1).bmp
